In [ ]:
print("ok")

In [23]:
import os

In [24]:
from __future__ import annotations

import os
from time import sleep

import leetcode

# Initialize client
configuration = leetcode.Configuration()

# NOTE: cookies var is just a dict with `csrftoken` and `LEETCODE_SESSION`
# fields which contain corresponding cookies from web browser
leetcode_session: str = os.environ["LEETCODE_SESSION"]

csrf_token = os.environ["CSRF_TOKEN"]

configuration.api_key["x-csrftoken"] = csrf_token
configuration.api_key["csrftoken"] = csrf_token
configuration.api_key["LEETCODE_SESSION"] = leetcode_session
configuration.api_key["Referer"] = "https://leetcode.com"
configuration.debug = False

api_instance = leetcode.DefaultApi(leetcode.ApiClient(configuration))


In [25]:
code = """
class Solution {
public:
    void bfs(int prevcolor,int row,int col,int color,vector<vector<int>>&visited,vector<vector<int>>&image)
    {
        // it'll ease the work
        queue<pair<int,int>> q;
        q.push({row,col});
        visited[row][col]=1;

        while(!q.empty())
        {
            int r = q.front().first;
            int c = q.front().second;
            q.pop();

            if(image[r][c]==prevcolor) 
              image[r][c]=color;
            
            // up
            if(r-1>=0 && !visited[r-1][c] && image[r-1][c]==prevcolor) 
              {q.push({r-1,c}); visited[r-1][c]=1;image[r-1][c]=color;}
            // right
            if(c+1<image[0].size() &&  !visited[r][c+1] && image[r][c+1]==prevcolor) 
              {q.push({r,c+1}); visited[r][c+1]=1;image[r][c+1]=color;}
            //down
            if(r+1<image.size() && !visited[r+1][c] && image[r+1][c]==prevcolor) 
              {q.push({r+1,c}); visited[r+1][c]=1; image[r+1][c]=color;}

            // left
            if(c-1>=0 && !visited[r][c-1] && image[r][c-1]==prevcolor) 
              {q.push({r,c-1}); visited[r][c-1]=1;image[r][c-1]=color;}
        }
    }
    vector<vector<int>> floodFill(vector<vector<int>>& image, int sr, int sc, int color) {
          int prevcolor = image[sr][sc];

          // visiting array.
          vector<vector<int>> visited(image.size(),vector<int>(image[0].size(),0));

          bfs(prevcolor,sr,sc,color,visited,image);

          return image;
    }
};
"""

In [ ]:
submission = leetcode.Submission(
    judge_type="large", typed_code=code, question_id=733, test_mode=False, lang="cpp"
)

submission_id = api_instance.problems_problem_submit_post(
    problem="flood-fill", body=submission
)


In [ ]:
print("Submission has been queued. Result:")
print(submission_id)

sleep(5)  # FIXME: should probably be a busy-waiting loop

submission_result = api_instance.submissions_detail_id_check_get(
    id=submission_id.submission_id
)

In [ ]:
print("Got submission result:")
print(submission_result)


## use selenium to get the cookiee

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import time

# --- Setup Chrome driver ---
options = Options()
options.add_argument(
    "--start-maximized"
)  # remove "--headless" so you can log in manually
driver = webdriver.Chrome(service=Service(), options=options)

# --- Open LeetCode Login Page ---
driver.get("https://leetcode.com/accounts/login/")

# --- Wait for manual login ---
print(
    "🔐 Please log in manually in the browser window (Google, GitHub, or credentials)..."
)
time.sleep(60)  # Give yourself time to log in

# --- Grab cookies ---
cookies = driver.get_cookies()

# --- Extract required cookies ---
leetcode_session = None
csrf_token = None

for cookie in cookies:
    if cookie["name"] == "LEETCODE_SESSION":
        leetcode_session = cookie["value"]
    elif cookie["name"] == "csrftoken":
        csrf_token = cookie["value"]

print("\n✅ Cookie values retrieved:")
print("LEETCODE_SESSION:", leetcode_session)
print("csrftoken:", csrf_token)

# Optionally save cookies for later use
# import pickle
# with open("leetcode_cookies.pkl", "wb") as f:
#     pickle.dump(cookies, f)

driver.quit()
